In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

FILE_PATH = 'your_file_name.csv' #
MODEL_FILE_NAME = 'model_raport.pkl'
RESULTS_FILE_NAME = 'raport_modelu.txt'


# --- 1. Wczytanie Danych z Pliku CSV ---
try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"BŁĄD: Nie znaleziono pliku w lokalizacji: {FILE_PATH}")
    exit()

# --- 2. Czyszczenie Danych (Scalanie Zduplikowanych Kolumn) ---
df_cleaned = df.groupby(level=0, axis=1).max()

# --- 3. Przygotowanie Danych (Preprocessing) ---
y = df_cleaned['Recruiter Decision']
X = df_cleaned.drop(['Resume_ID', 'Recruiter Decision'], axis=1)

# --- 4. Podział na Zbiór Treningowy i Testowy ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

# --- 5. Trenowanie Modelu (Random Forest) ---
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)
print("--- 5. Model został pomyślnie wytrenowany ---")


# --- 6. Ocena Modelu (Ewaluacja) ---
y_pred = model.predict(X_test)

# Przygotowujemy stringi do zapisu do pliku
accuracy_str = f"Dokładność (Accuracy): {accuracy_score(y_test, y_pred):.2f}"
report_str = classification_report(y_test, y_pred, target_names=['Odrzucony (0)', 'Zaproszony (1)'], zero_division=0)


# --- 7. Ważność Cech (Co jest najważniejsze dla modelu?) ---
importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame(
    {'Cecha': feature_names, 'Ważność': importances}
).sort_values(by='Ważność', ascending=False)

importance_str = feature_importance_df.head(10).to_string()


# --- 8. ZAPIS MODELU I WYNIKÓW DO PLIKU ---
print(f"\n--- 8. Zapisywanie wyników i modelu ---")

# Zapis raportu tekstowego (Krok 6 i 7)
try:
    with open(RESULTS_FILE_NAME, 'w', encoding='utf-8') as f:
        f.write("--- 6. Ewaluacja modelu na danych testowych ---\n")
        f.write(accuracy_str + "\n\n")
        f.write("Raport Klasyfikacji:\n")
        f.write(report_str + "\n\n")
        f.write("--- 7. Ważność Cech (Feature Importance) ---\n")
        f.write("10 najważniejszych cech wpływających na decyzję:\n")
        f.write(importance_str)
    
    print(f"Pomyślnie zapisano raport do pliku: {RESULTS_FILE_NAME}")

except IOError as e:
    print(f"BŁĄD: Nie można zapisać raportu do pliku {RESULTS_FILE_NAME}. Błąd: {e}")

# Zapis modelu binarnego
try:
    joblib.dump(model, MODEL_FILE_NAME)
    print(f"Pomyślnie zapisano model do pliku: {MODEL_FILE_NAME}")
except Exception as e:
    print(f"BŁĄD: Nie można zapisać modelu do pliku {MODEL_FILE_NAME}. Błąd: {e}")


# --- 9. Przykładowa Predykcja na Nowych Danych (jak wcześniej) ---
# Ten krok pozostaje bez zmian, używa modelu z pamięci
nowy_kandydat_dane = {
    'Experience (Years)': [5], 'Education': [2], 'Certifications': [1], 
    'Job Role': [1], 'Salary Expectation ($)': [90000], 'Projects Count': [4], 
    'C++': [0], 'Cybersecurity': [0], 'Deep Learning': [1], 
    'Ethical Hacking': [0], 'Java': [0], 'Linux': [1], 
    'Machine Learning': [1], 'NLP': [0], 'Networking': [0], 
    'Python': [1], 'Pytorch': [0], 'React': [0], 'SQL': [1], 'TensorFlow': [1]
}
nowy_kandydat_df = pd.DataFrame(nowy_kandydat_dane, columns=X.columns).fillna(0)
prawdopodobienstwo = model.predict_proba(nowy_kandydat_df)
# print(f"\nSzansa na zaproszenie (z modelu w pamięci): {prawdopodobienstwo[0][1] * 100:.2f}%")


# --- 10. PRZYKŁAD WCZYTANIA ZAPISANEGO MODELU I UŻYCIA GO ---
print(f"\n--- 10. Test wczytania i użycia zapisanego modelu ---")

try:
    # Wczytujemy model z pliku
    wczytany_model = joblib.load(MODEL_FILE_NAME)
    print(f"Pomyślnie wczytano model z pliku: {MODEL_FILE_NAME}")
    
    # Używamy wczytanego modelu na tych samych danych
    nowa_predykcja = wczytany_model.predict_proba(nowy_kandydat_df)
    
    print(f"\nPredykcja dla 'nowego kandydata' przy użyciu WCZYTANEGO modelu:")
    print(f"Prawdopodobieństwo (Odrzuć, Zaproś): {nowa_predykcja[0]}")
    print(f"Szansa na zaproszenie: {nowa_predykcja[0][1] * 100:.2f}%")
    
    # Sprawdzenie czy wyniki są takie same
    if np.array_equal(prawdopodobienstwo, nowa_predykcja):
        print("Weryfikacja: Wynik z wczytanego modelu jest IDYCZNY jak z modelu w pamięci.")
    
except FileNotFoundError:
    print(f"BŁĄD: Nie można wczytać modelu. Plik {MODEL_FILE_NAME} nie istnieje.")
except Exception as e:
    print(f"BŁĄD: Wystąpił problem przy wczytywaniu modelu. Błąd: {e}")

--- 5. Model został pomyślnie wytrenowany ---

--- 8. Zapisywanie wyników i modelu ---
Pomyślnie zapisano raport do pliku: raport_modelu.txt
Pomyślnie zapisano model do pliku: model_raport.pkl

--- 10. Test wczytania i użycia zapisanego modelu ---
Pomyślnie wczytano model z pliku: model_raport.pkl

Predykcja dla 'nowego kandydata' przy użyciu WCZYTANEGO modelu:
Prawdopodobieństwo (Odrzuć, Zaproś): [0.89 0.11]
Szansa na zaproszenie: 11.00%
Weryfikacja: Wynik z wczytanego modelu jest IDYCZNY jak z modelu w pamięci.


/var/folders/fr/xf8zmfr90fsfv_zpmb43xwb40000gn/T/ipykernel_29367/2072269599.py:23: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_cleaned = df.groupby(level=0, axis=1).max()
/var/folders/fr/xf8zmfr90fsfv_zpmb43xwb40000gn/T/ipykernel_29367/2072269599.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nowy_kandydat_df = pd.DataFrame(nowy_kandydat_dane, columns=X.columns).fillna(0)
